In [1]:
# !jupyter nbconvert --to script AFL.ipynb

In [2]:
# !pip install pyomo
# !pip install --upgrade --user ortools
# !apt-get install -y glpk-utils
# !sudo apt-get update -y
# !sudo apt-get install -y coinor-cbc

In [3]:
# !pip install -U git+https://github.com/coin-or/pulp
# !sudo pulptest
# !pip install pandas

In [13]:
import numpy as np
import pandas as pd
from pulp import *

In [14]:
def test1(test_df):
    assert len(test_df[test_df['Name'].isna()])==0
    for r in test_df['Round'].unique():
        names = test_df[test_df['Round']==r]['Name'].unique()
        try:    
            assert len(test_df[~(test_df['Name'].isin(names))])==0
        except AssertionError:
            # print(r)
            # print(test_df[~(test_df['Name'].isin(names))])
            assert len(test_df[~(test_df['Name'].isin(names))])==0

In [15]:
def getPlayers(full_player_scores = pd.read_csv('full_players.csv', index_col=0)):
    full_player_scores = pd.read_csv('full_players.csv')
    full_player_scores = full_player_scores.dropna(subset=['Name'])
    full_player_scores = full_player_scores.reset_index(drop=True)
    full_player_scores['Score'] = pd.to_numeric(full_player_scores['Score'], errors='coerce')

    # testfilter
    full_player_scores = full_player_scores[full_player_scores['Round']<3]

    full_player_scores["Value"] = full_player_scores['Score']/full_player_scores['Price']
    players_to_be_kept = full_player_scores.groupby('Name').mean()[['Score','Value']]

    players_to_keep = []
    # num_players_to_keep = int(len(players_to_be_kept)*0.2)
    num_players_to_keep = int(len(players_to_be_kept)*0.1)
    players_to_keep += list(players_to_be_kept.sort_values('Score', ascending = False).index[:num_players_to_keep])
    players_to_keep += list(players_to_be_kept.sort_values('Value', ascending = False).index[:num_players_to_keep])

    full_player_scores = full_player_scores[full_player_scores['Name'].isin(players_to_keep)]

    full_player_scores_2021 = full_player_scores[full_player_scores['Year']==2021]
    full_player_scores['Score'] = full_player_scores[['Name','Team 1', 'Team 2', 'Team', 'Position','Score']].groupby(['Name','Team 1', 'Team 2', 'Position']).transform('mean')
    # full_player_scores = full_player_scores[full_player_scores['Year']==2020]

    schedule = pd.read_csv('2021_schedule', index_col=0) 
    schedule = schedule.append(schedule.rename({'Team 1':"Team 2", "Team 2":"Team 1"},axis=1))
    full_player_scores = schedule.merge(full_player_scores[['Team 1', 'Team 2', 'Position', 'Name', 'Score', 'Price']], on=['Team 1','Team 2'])
    full_player_scores = full_player_scores.drop_duplicates()

    # Only keep players who are playing this season
    full_player_scores = full_player_scores[full_player_scores['Name'].isin(full_player_scores_2021['Name'].unique())].dropna(how='all').dropna(how='all')

    df = full_player_scores.copy()

    df1 = df[['Name','Team 1','Position']].drop_duplicates()
    df2 = pd.DataFrame(range(1,23), columns=['Round'])
    df1['key'] = 0
    df2['key'] = 0
    df = df1.merge(df2, how='outer', on=['key'])
    df = df.sort_values('Round')
    df = df.drop('key', axis=1)
    full_player_scores = full_player_scores.merge(df, on=['Name', 'Team 1', 'Position','Round'],how='right').reset_index(drop=True)
    try:
        test1(full_player_scores)
    except AssertionError:
        print('WARNING: failed a test')

    player_num_games_played = full_player_scores[full_player_scores['Score']>0].groupby('Name')['Team 2'].transform('count')

    opponent_average_score = full_player_scores.groupby('Team 2')['Score'].transform('mean')
    player_average_score = full_player_scores.groupby('Name')['Score'].transform('mean')
    player_average_value = player_average_score/full_player_scores['Price']

    na_locs = full_player_scores['Score'].isna()

    full_player_scores.loc[na_locs,'Score'] = ((opponent_average_score+player_average_score)/2).loc[na_locs]

    na_locs = full_player_scores['Score'].isna()

    full_player_scores.loc[na_locs,'Score'] = (player_average_score).loc[na_locs]

    RU_Players = full_player_scores.copy()


    RU_Players = RU_Players.sort_values('Score',ascending=False)

    RU_Players = full_player_scores.copy()


    RU_Players = RU_Players.sort_values('Score',ascending=False)
    RU_Players['id'] = "R:"+RU_Players['Round'].astype(str)+"_P:"+RU_Players['Name']

    # Only take players who played more than 5 games
    # players_to_keep = player_num_games_played[player_num_games_played>5].index.values
    # RU_Players = RU_Players.reindex(index=players_to_keep).dropna(how='all')

    try:
        test1(RU_Players)
    except AssertionError:
        print('WARNING: failed a test')

    RU_Players.groupby(['Round','Position']).count()
    return RU_Players
RU_Players = getPlayers()
RU_Players = RU_Players.replace(np.nan, 0)

In [20]:
def getProblem(RU_Players):
  player_contraints = {}
  prob = LpProblem("aflProblem", LpMaximize)
  overall_score = LpVariable('OverallScore',0)
  for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems()):
    p_id = p_id[1]
    player_contraints[p_id] = LpVariable(p_id, 0, 1, cat='Binary')

  player_contraints = LpVariable.dicts("PC", player_contraints, 0, 1, cat='Binary')
  prob += lpSum([player_contraints[p_id[1]]*score[1] for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems())]), "Total score is maximized"

  # START TRANFERS
  RU_Players_prev_round = RU_Players
  RU_Players_prev_round['Round_prev'] = RU_Players_prev_round['Round'] - 1

  RU_transfers = RU_Players[['id','Position','Round','Name']].merge(RU_Players_prev_round[['id','Position','Round_prev','Name']],left_on=['Position','Round'],right_on=['Position','Round_prev'], suffixes = ('_prev','_next'))

  RU_transfers['Transfer'] = RU_transfers['id_prev']+'-'+RU_transfers['id_next']
  RU_transfers=RU_transfers[['id_prev',	'Round',	'Name_prev',	'id_next',	'Round_prev',	'Name_next',	'Transfer']].drop_duplicates()

  transfer_contraints = {}

  # Define transfer itermediaries
  for i,t_id in RU_transfers['Transfer'].iteritems():
    transfer_contraints[t_id] = LpVariable(t_id, 0, 1, cat='Binary')

  transfer_contraints = LpVariable.dicts("TC", transfer_contraints, 0, 1, cat='Binary')

  # map rounds end to itermediary
  for prev_player,trans in RU_transfers.groupby(['id_prev'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[prev_player], f"Previous player equals transfer intermediatary for {prev_player} equals {trans}"

  # map intermediary to next round
  for next_player,trans in RU_transfers.groupby(['id_next'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[next_player], f"Next player equals transfer intermediatary for {next_player} equals {trans}"

  for round, transfers in RU_transfers[RU_transfers['Name_prev'] != RU_transfers['Name_next']][['Round','Transfer']].drop_duplicates().groupby(['Round'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in transfers]) <= 2, f"Round: {round}, has less than or equal to 4 transfers"
    # print(f'{[transfer_contraints[t_id] for t_id in transfers]}) <= 2')
  # END TRANFERS

  # START money contraint
  money = 13 * 10**6
  for round,player in RU_Players[['id','Price','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
    prob += lpSum([player_contraints[p_id]*RU_Players[RU_Players['id']==p_id]['Price'].values[0] for p_id in player]) <= money, f"Round: {round}, has less than ${money}"
  # END money contraint

  #START Contrain each player to a single position per round
  RU_Players_Position_Casting = RU_Players.copy()
  RU_Players_Position_Casting['pl_pos_id'] = (RU_Players['id']+'_'+RU_Players['Position'])

  Players_Position_Casting_Series = RU_Players_Position_Casting[['id','pl_pos_id','Round']].drop_duplicates().groupby(['id','Round'])['pl_pos_id'].apply(list).iteritems()

  Players_Position_Casting = {}
  for i,r in RU_Players_Position_Casting[['pl_pos_id']].drop_duplicates().iterrows():
    Players_Position_Casting[r['pl_pos_id']] = LpVariable(r['pl_pos_id'], 0, 1, cat='Binary')

  Players_Position_Casting = LpVariable.dicts("Players_Position_Casting", Players_Position_Casting, 0, 1, cat='Binary')

  for (p_id,round), pl_pos_ids in Players_Position_Casting_Series:
      player_possible_positions = [Players_Position_Casting[pl_pos_id] for pl_pos_id  in pl_pos_ids] 
      prob += lpSum(player_possible_positions) <= player_contraints[p_id], f"Must have only have equal to {p_id} (0 or 1) of {player_possible_positions} positions in round {round}"
  #END Contrain each player to a single position per round

  # START max players from each position
  allowed_holds_per_position = {'DE': 8, "MI" : 10, 'RU' : 3, 'FO':9}

  print('START max players from each position')
  for (position,round), pl_pos_ids in RU_Players_Position_Casting[['pl_pos_id','Position','Round']].drop_duplicates().groupby(['Position', 'Round'])['pl_pos_id'].apply(list).iteritems():
    prob += lpSum([Players_Position_Casting[pl_pos_id] for pl_pos_id in pl_pos_ids]) == allowed_holds_per_position[position], f"Position: {position}, has less than {allowed_holds_per_position[position]} in round {round}"
  # END max players from each position

  print('done with pre-work')
  return prob, transfer_contraints, Players_Position_Casting, RU_Players_Position_Casting
prob, transfer_contraints,Players_Position_Casting, RU_Players_Position_Casting = getProblem(RU_Players)

START max players from each position
done with pre-work


In [21]:
try:
    solver = getSolver('COIN_CMD', timeLimit=4000, msg=True,gapRel = 0.2)
    prob.solve(solver)
except:
    print('USING INBUILT SOLVER!!  \n')
    solver = getSolver('PULP_CBC_CMD')
    prob.solve(solver)

In [22]:
prob.status

1

In [23]:
for i,v in transfer_contraints.items():
    if v.value() != 0:
        last_name = i[i.find(' ')+1: i.find('-')]
        last_letters = i[-len(last_name):]
        if not last_name == last_letters:
            print(i)

R:2_P:Dayne Zorko-R:3_P:Lachlan Hunter
R:2_P:Shaun Higgins-R:3_P:Andrew Gaff
R:1_P:Sebastian Ross-R:2_P:Scott Pendlebury
R:1_P:Callan Ward-R:2_P:Jackson Macrae


In [24]:
results = []
for name, player_position in Players_Position_Casting.items():
    if player_position.value() != 0:
        results.append([name,1])
Players_selected = pd.DataFrame(results, columns=['pl_pos_id', 'is_selected'])
RU_Players_Position_Casting = RU_Players_Position_Casting.merge(Players_selected, on = ['pl_pos_id'])
RU_Players_Position_Casting.to_csv('solution.csv')
print(RU_Players_Position_Casting.groupby(['Round','Position']).sum()['is_selected'])

Round  Position
1      DE           8
       FO           9
       MI          10
       RU           3
2      DE           8
       FO           9
       MI          10
       RU           3
3      DE           8
       FO           9
       MI          10
       RU           3
Name: is_selected, dtype: int64


In [14]:
# !git add solution.csv
# !git commit -m "sol"
# !git push

[main e609153] sol
 Committer: Ubuntu <ubuntu@ip-172-31-2-142.ap-southeast-2.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 741 insertions(+), 545 deletions(-)
 rewrite solution.csv (98%)
Username for 'https://github.com': ^C


In [10]:
RU_Players_Position_Casting.sort_values(['Round'])

,Round,Team 1,Team 2,Position,Name,Score,Price,id,Round_prev,pl_pos_id,is_selected
581,1,Bombers,0,DE,Orazio Fantasia,65.666667,0.0,R:1_P:Orazio Fantasia,0,R:1_P:Orazio Fantasia_DE,1
79,1,Cats,0,MI,Joel Selwood,97.695455,0.0,R:1_P:Joel Selwood,0,R:1_P:Joel Selwood_MI,1
502,1,Demons,0,RU,Braydon Preuss,77.500000,0.0,R:1_P:Braydon Preuss,0,R:1_P:Braydon Preuss_RU,1
616,1,Bombers,0,FO,Jake Stringer,63.476190,0.0,R:1_P:Jake Stringer,0,R:1_P:Jake Stringer_FO,1
619,1,Hawks,0,DE,James Frawley,60.850000,0.0,R:1_P:James Frawley,0,R:1_P:James Frawley_DE,1
...,...,...,...,...,...,...,...,...,...,...,...
247,22,Kangaroos,0,RU,Todd Goldstein,89.200000,0.0,R:22_P:Todd Goldstein,21,R:22_P:Todd Goldstein_RU,1
595,22,Kangaroos,0,FO,Taylor Garner,64.750000,0.0,R:22_P:Taylor Garner,21,R:22_P:Taylor Garner_FO,1
593,22,Saints,0,DE,Jarryn Geary,64.750000,0.0,R:22_P:Jarryn Geary,21,R:22_P:Jarryn Geary_DE,1
418,22,Hawks,0,DE,Daniel Howe,83.250000,0.0,R:22_P:Daniel Howe,21,R:22_P:Daniel Howe_DE,1


In [9]:
for round in RU_Players_Position_Casting['Round'].unique():
    assert RU_Players_Position_Casting[RU_Players_Position_Casting['Round']==round]['is_selected'].sum()